In [ ]:
import csv
import random
import classes
import crossover
import pandas as pd

from deap import base
from deap import creator
from deap import tools

In [ ]:
# Get data from csv file
def get_data(csv_file):
  data = []
  with open(csv_file, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip first line (headers)
    for row in csv_reader:
      punto = row[0]
      latitude = float(row[1])
      longitude = float(row[2])
      address = row[3]
      data.append((punto, latitude, longitude, address))
  return data

In [ ]:
def shuffle_pos(pos):
  random.shuffle(pos)
  return pos

In [ ]:
points_of_sale = get_data('data.csv')
# Point_of_Sale list
pos = []
for point in points_of_sale:
  pos.append(classes.Point_of_Sale(point[0], point[1], point[2], point[3]))

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_sequence", shuffle_pos, pos)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_sequence)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

population = toolbox.population(n=5)

In [ ]:
pop_df = pd.DataFrame(population)
pop_df

In [ ]:
#https://deap.readthedocs.io/en/master/api/tools.html#id2
#https://en.wikipedia.org/wiki/Crossover_(genetic_algorithm)
#file:///C:/Users/llaaj/AppData/Local/Temp/MicrosoftEdgeDownloads/b9e8d235-e495-4ff4-86e8-6678782e06ba/download.pdf
toolbox.register("mate", crossover.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.1)
# tournsize = The number of individuals participating in each tournament
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", crossover.evaluate)

In [ ]:
def main():
  mate_probability = 0.5
  mutation_probability = 0.2
  number_of_generations = 40
  population = toolbox.population(n=25)
  # Evaluate the entire population
  fitnesses = map(toolbox.evaluate, population)

  for individual, fitness in zip(population, fitnesses):
    individual.fitness.values = fitness

  for generation in range(number_of_generations):
    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))

    # Clone the selected individuals
    offspring = map(toolbox.clone, offspring)
    offspring = list(offspring
    )
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
      if random.random() < mate_probability:
        toolbox.mate(child1, child2)
        del child1.fitness.values
        del child2.fitness.values

    for mutant in offspring:
      if random.random() < mutation_probability:
        toolbox.mutate(mutant)
        del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
      ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    
    population[:] = offspring

  return population  


In [ ]:
pop = main()
pop_df = pd.DataFrame(pop)
pop_df

In [ ]:
x = crossover.print_responses()
pd.DataFrame.from_dict(crossover.routes_responses_dict)
